# Child Tax Credit

The Child Tax Credit (CTC) is a partially refundable tax credit for filers with children that phases in and out with income.


In [1]:
from policyengine_us.tools.documentation import (
    variable_summary,
    variation_chart,
)

variable_summary("ctc")


Unable to load OpenFisca variables from file "/Users/maxghenis/PolicyEngine/policyengine-us/policyengine_us/variables/contrib/ubi_center/flat_tax.py"


AttributeError: module 'sortedcontainers' has no attribute 'sorteddict'

In [ ]:
variation_chart("ctc")


In [ ]:
from policyengine_us import IndividualSim
import pandas as pd
import plotly.express as px

LIGHT_GRAY = "#F5F5F5"
GRAY = "#BDBDBD"
BLUE = "#5091cc"
LIGHT_BLUE = "lightblue"
DARK_BLUE = "darkblue"

COLOR_MAP = {"0": GRAY, "1": LIGHT_BLUE, "2": BLUE, "3": DARK_BLUE}


def make_ctc(adults, children):
    sim = IndividualSim(year=2022)
    sim.add_person(name="head", age=25)
    members = ["head"]
    if adults == 2:
        sim.add_person(name="spouse")
        members += ["spouse"]
    for i in range(children):
        child = "child{}".format(i)
        sim.add_person(name=child, age=6)
        members += [child]
    sim.add_tax_unit(name="tax_unit", members=members)
    sim.add_household(name="household", members=members)
    sim.vary("employment_income", max=60_000, step=100)
    return pd.DataFrame(
        dict(
            employment_income=sim.calc("employment_income")[0],
            ctc=sim.calc("ctc")[0].round(),
            mtr=-sim.deriv("ctc", "employment_income", wrt_target="head"),
            adults=adults,
            children=str(children),
        )
    )


# Make a table of CTC amounts for different numbers of adults and children.
l = []
for adults in range(1, 3):
    for children in range(0, 4):
        l.append(make_ctc(adults, children))

df = pd.concat(l)

LABELS = dict(
    employment_income="Employment income",
    income_source="Income source",
    ctc="Child tax credit",
    mtr="CTC marginal tax rate",
    children="Children",
    adults="Adults",
)

fig = px.line(
    df,
    "employment_income",
    "ctc",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Child tax credit, 2022",
    color_discrete_map=COLOR_MAP,
)

fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
)
fig.show()


Now compute the net income impact of un-repealing the CTC.


In [2]:
from policyengine_us import IndividualSim
from policyengine_us.model_api import *
import pandas as pd
import plotly.express as px

LIGHT_GRAY = "#F5F5F5"
GRAY = "#BDBDBD"
BLUE = "#5091cc"
LIGHT_BLUE = "lightblue"
DARK_BLUE = "darkblue"

COLOR_MAP = {"0": GRAY, "1": LIGHT_BLUE, "2": BLUE, "3": DARK_BLUE}


def make_net_income(adults, children, year, reform):
    if reform is None:
        sim = IndividualSim(year=year)
    else:
        sim = IndividualSim(reform, year=year)
    sim.add_person(name="head", age=25)
    members = ["head"]
    if adults == 2:
        sim.add_person(name="spouse")
        members += ["spouse"]
    for i in range(children):
        child = "child{}".format(i)
        sim.add_person(name=child, age=6)
        members += [child]
    sim.add_tax_unit(name="tax_unit", members=members)
    sim.add_household(name="household", members=members)
    sim.vary("employment_income", max=200_000, step=1_000)
    return pd.DataFrame(
        dict(
            employment_income=sim.calc("employment_income")[0],
            income_tax=sim.calc("income_tax"),
        )
    )


class neutralize_ctc(Reform):
    def apply(self):
        self.neutralize_variable("ctc")


def get_ctc_impact(adults, children, year):
    baseline = make_net_income(adults, children, year, None)
    reform = make_net_income(adults, children, year, neutralize_ctc)
    return pd.DataFrame(
        dict(
            adults=adults,
            children=children,
            year=year,
            employment_income=baseline.employment_income,
            baseline=baseline.income_tax,
            reform=reform.income_tax,
            ctc=reform.income_tax - baseline.income_tax,
        )
    )


# Make a table of CTC amounts for different numbers of adults and children.
l = []
for adults in range(1, 3):
    for children in range(0, 4):
        for year in [2020, 2021, 2022]:
            l.append(get_ctc_impact(adults, children, year))

df = pd.concat(l)

LABELS = dict(
    employment_income="Employment income",
    income_source="Income source",
    ctc="Child tax credit",
    mtr="CTC marginal tax rate",
    children="Children",
    adults="Adults",
)

fig = px.line(
    df,
    "employment_income",
    "ctc",
    color="children",
    facet_col="adults",
    animation_frame="year",
    labels=LABELS,
    title="Child tax credit",
    color_discrete_map=COLOR_MAP,
)

fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
)
fig.show()


Unable to load OpenFisca variables from file "/Users/maxghenis/PolicyEngine/policyengine-us/policyengine_us/variables/contrib/ubi_center/flat_tax.py"


AttributeError: module 'sortedcontainers' has no attribute 'sorteddict'